## Packages à installer :

In [ ]:
#pip install torch spacy transformers pandas numpy

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
#df = pd.read_csv('../scraping/scraping_gse.csv', sep='|') 
# text = str(df['article_content'][0])


In [ ]:
from transformers import pipeline

# Initialisez le pipeline pour l'analyse de sentiment
nlp = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device="cpu")

# Exemple de texte en français
#text = "J'adore ce produit ! C'est absolument incroyable. " * 100  # Texte long pour l'exemple

# Diviser le texte en segments de 512 tokens
max_length = 512
segments = [text[i:i + max_length] for i in range(0, len(text), max_length)]

# Analyse sentimentale pour chaque segment
results = [nlp(segment) for segment in segments]

# Afficher les résultats
for i, result in enumerate(results):
    print(f"Segment {i + 1}: {result}")

In [ ]:
from os import listdir
fichiers = [x for x in listdir('../scraping') if x.endswith('.csv') and x.startswith('scraping_')]
df_list = []
for x in tqdm(fichiers):
    file_path = f'../scraping/{x}'
    df_csv = pd.read_csv(file_path, sep='|')
    df_list.append(df_csv)
# Concaténer les dataframes de la liste
df_final = pd.concat(df_list)
# Réinitialiser l'index du dataframe final
df_final = df_final.reset_index(drop=True)

In [ ]:
df_final.loc[df_final['website'] == 'https://www.droit-compta-gestion.fr']

In [ ]:
#pip install jupyter ipywidgets nltk matplotlib seaborn plotly plotly-express wordcloud ydata-profiling

In [ ]:
import nltk
import re
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
#Assurez-vous de télécharger les ressources nécessaires de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
df = df_final
len(df)

#Exemple de texte
#text = my_df["article_content"][0]
def get_keywords(my_df):
    keywords = []
    keyword_rank = []
    url_list = []
    website_list = []

    for i in tqdm(range(len(my_df))):
        cleaned_text = clean_text(my_df['article_content'][i])

        # Tokenisation et suppression des stop words
        stop_words = set(stopwords.words('french'))
        tokens = word_tokenize(cleaned_text)
        filtered_tokens = [word for word in tokens if word not in stop_words]

        # Lemmatisation
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

        # Vérification des tokens lemmatisés
        # print(lemmatized_tokens)

        # Fréquence des mots
        word_freq = Counter(lemmatized_tokens)
        
        for j in range(len(word_freq)):
            keywords.append(word_freq.most_common()[j][0])
            keyword_rank.append(word_freq.most_common()[j][1])
            url_list.append(my_df['article_url'][i])
            website_list.append(my_df['website'][i])
        
    return pd.DataFrame({
        'article_url': url_list,
        'keywords': keywords,
        'keyword_rank': keyword_rank,
        'website': website_list
    })
        

In [ ]:
import nltk
import re
from collections import Counter
import pandas as pd
from tqdm.notebook import tqdm
import spacy

# Assurez-vous de télécharger le modèle français de spacy
# python -m spacy download fr_core_news_sm

nlp = spacy.load('fr_core_news_sm')

#Nettoyage du texte
def clean_text(text):
    text = re.sub(r'\s+', ' ', str(text))  # Suppression des espaces multiples
    text = re.sub(r'[^\w\s]', '', str(text))  # Suppression de la ponctuation
    text = re.sub(r'\d', '', text)  # Suppression des caractères numériques
    text = text.lower()  # Conversion en minuscules
    return text if len(text) >= 2 else ''

def lemmatize_french(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

def get_keywords(my_df):
    keywords = []
    keyword_rank = []
    url_list = []
    website_list = []

    stop_words = set(stopwords.words('french'))

    for i in tqdm(range(len(my_df))):
        cleaned_text = clean_text(my_df['article_content'][i])
        tokens = word_tokenize(cleaned_text)
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words and len(word) > 2]
        lemmatized_tokens = lemmatize_french(filtered_tokens)
        word_freq = Counter(lemmatized_tokens)

        for word, freq in word_freq.items():
            keywords.append(word)
            keyword_rank.append(freq)
            url_list.append(my_df['article_url'][i])
            website_list.append(my_df['website'][i])

    return pd.DataFrame({
        'article_url': url_list,
        'keywords': keywords,
        'keyword_rank': keyword_rank,
        'website': website_list
    })

In [ ]:
df.to_csv('raw_scraping.csv', sep='|')

In [ ]:
df.columns

In [ ]:
df_new = get_keywords(df)

In [ ]:
df_new

In [ ]:
df_new.to_csv('keywords_analysis.csv', index=False)

In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(df_new)

In [ ]:
df_new['website'].value_counts()

In [ ]:
df_new

In [ ]:
import pandas as pd
import spacy
from tqdm import tqdm

# Charger le modèle français
nlp = spacy.load('fr_core_news_sm')

def extract_keywords(text):
    # Traiter le texte avec spaCy
    doc = nlp(text)
    
    # Extraire les mots-clés : noms, adjectifs, et entités nommées
    keywords = set()
    for token in doc:
        if token.pos_ in {'NOUN', 'PROPN', 'ADJ'}:  # Noms, noms propres, adjectifs
            keywords.add(token.lemma_)
    
    # Ajouter les entités nommées
    for ent in doc.ents:
        keywords.add(ent.text)
    
    return list(keywords)

# Appliquer la fonction à chaque ligne du DataFrame
tqdm.pandas(desc="Extraction des mots-clés")
df['keywords'] = df['article_content'].progress_apply(extract_keywords)

# Afficher les résultats
print(df[['article_content', 'keywords']].head())

In [ ]:
import nltk
import re
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from tqdm.notebook import tqdm
import spacy

# Assurez-vous de télécharger les ressources nécessaires de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Charger le modèle français de spaCy
nlp = spacy.load('fr_core_news_sm')

# Nettoyage du texte
def clean_text(text):
    text = re.sub(r'\s+', ' ', str(text))  # Suppression des espaces multiples
    text = re.sub(r'[^\w\s]', '', str(text))  # Suppression de la ponctuation
    text = re.sub(r'\d', '', text)  # Suppression des caractères numériques
    text = text.lower()  # Conversion en minuscules
    return text if len(text) >= 2 else ''

# Fonction pour extraire les mots-clés avec spaCy
def extract_keywords(text):
    # Traiter le texte avec spaCy
    doc = nlp(text)
    
    # Extraire les mots-clés : noms, adjectifs, et entités nommées
    keywords = set()
    for token in doc:
        if token.pos_ in {'NOUN', 'PROPN', 'ADJ'}:  # Noms, noms propres, adjectifs
            keywords.add(token.lemma_)
    
    # Ajouter les entités nommées
    for ent in doc.ents:
        keywords.add(ent.text)
    
    return list(keywords)

# Fonction pour obtenir les mots-clés de notre DataFrame
def get_keywords(my_df):
    keywords = []
    keyword_rank = []
    url_list = []
    website_list = []

    for i in tqdm(range(len(my_df))):
        cleaned_text = clean_text(my_df['article_content'][i])

        # Extraire les mots-clés avec spaCy
        extracted_keywords = extract_keywords(cleaned_text)

        # Fréquence des mots
        word_freq = Counter(extracted_keywords)
        
        for word, freq in word_freq.items():
            keywords.append(word)
            keyword_rank.append(freq)
            url_list.append(my_df['article_url'][i])
            website_list.append(my_df['website'][i])
        
    return pd.DataFrame({
        'article_url': url_list,
        'keywords': keywords,
        'keyword_rank': keyword_rank,
        'website': website_list
    })

# Afficher les résultats
keyword_df.head()

In [ ]:
import nltk
import re
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from tqdm.notebook import tqdm
import spacy
from spacy import displacy

# Assurez-vous de télécharger les ressources nécessaires de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords

# Charger le modèle français de spaCy
nlp = spacy.load('fr_core_news_sm')

# Charger les stop words en français
stop_words = set(stopwords.words('french'))

# Nettoyage du texte
def clean_text(text):
    text = re.sub(r'\s+', ' ', str(text))  # Suppression des espaces multiples
    text = text.lower()  # Conversion en minuscules

    # Séparer les mots et conserver la partie droite après l'apostrophe
    words = text.split()
    words = [word.split("’")[-1] for word in words]  # Conserver la partie droite des mots avec apostrophes

    # Rejoindre les mots nettoyés
    text = ' '.join(words)
    
    text = re.sub(r"[^\w\s]", '', str(text))  # Suppression de la ponctuation sauf les apostrophes
    text = re.sub(r'\d', '', text)  # Suppression des caractères numériques
    
    
    return text if len(text) >= 2 else ''

# Fonction pour extraire les mots-clés avec spaCy
def extract_keywords(text):
    # Traiter le texte avec spaCy
    doc = nlp(text)
    
# Extraire les mots-clés : noms, adjectifs, et entités nommées
    keywords = []
    for chunk in doc.noun_chunks:
        # Filtrer les stop words des noun chunks
        filtered_chunk = ' '.join([word for word in chunk.text.split() if word.lower() not in stop_words])
        if filtered_chunk:
            keywords.append(filtered_chunk)
    
    # Ajouter les entités nommées
    for ent in doc.ents:
        keywords.append(ent.text)
    
    return keywords

# Fonction pour obtenir les mots-clés d'un DataFrame
def get_keywords(my_df):
    keywords = []
    keyword_rank = []
    url_list = []
    website_list = []

    for i in tqdm(range(len(my_df))):
        cleaned_text = clean_text(my_df['article_content'][i])

        # Extraire les mots-clés avec spaCy
        extracted_keywords = extract_keywords(cleaned_text)

        # Fréquence des mots
        word_freq = Counter(extracted_keywords)
        
        for word, freq in word_freq.items():
            keywords.append(word)
            keyword_rank.append(freq)
            url_list.append(my_df['article_url'][i])
            website_list.append(my_df['website'][i])

    return pd.DataFrame({
        'article_url': url_list,
        'keywords': keywords,
        'keyword_rank': keyword_rank,
        'website': website_list
    })

# Exemple d'utilisation avec votre DataFrame
keyword_df = get_keywords(df_final)

# Afficher les résultats
keyword_df.head()

In [ ]:
keyword_df.to_csv('keywords_analysis.csv', index=False)

for index, row in df.iterrows():
    texte = row['article_content']
    doc = nlp(texte)
    print(f"Texte {index + 1}:")
    displacy.render(doc, style='ent', jupyter=True)